In [1]:
# Load training data
from __future__ import print_function
import pickle 
import numpy as np
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable  

In [20]:
trainset_imoprt = pickle.load(open("train_labeled.p", "rb"))
validset_import = pickle.load(open("validation.p", "rb"))
train_loader = torch.utils.data.DataLoader(trainset_imoprt, batch_size=64, shuffle=True)
valid_loader = torch.utils.data.DataLoader(validset_import, batch_size=64, shuffle=True)

In [7]:
train_np = trainset_imoprt.train_data.numpy()
train_labels = trainset_imoprt.train_labels.numpy()

In [10]:
testimage = train_np[0]

In [55]:
# Auto_encoder model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28*28, 14*14)
        self.decoder = nn.Linear(14*14,28*28)

    def forward(self, x):
        #Change x dimension from 28*28 to 1*(28*28)
        batch,_,m,n = x.size()
        x = x.view(batch,m*n)
        x = F.sigmoid(self.fc1(x))
        return self.decoder(x)

model = Net()

In [56]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [102]:
# Training
# CPU only training
def train(epoch):
    model.train()
    for batch_idx, (data, label) in enumerate(train_loader):
        data, label = Variable(data), Variable(label)
        optimizer.zero_grad()
        output = model(data)
        batch,_,m,n = data.size()
        target = data.view(batch,m*n)
        diff = target-output
        loss = torch.mm(diff,diff.transpose(0,1)).diag()
        loss= torch.mean(loss)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))

def test(epoch, valid_loader):
    model.eval()
    test_loss = 0
    for data, label in valid_loader:

        data, label = Variable(data, volatile=True), Variable(label)
        output = model(data)
        batch,_,m,n = data.size()
        target = data.view(batch,m*n)
        diff = target-output
        loss = torch.mm(diff,diff.transpose(0,1)).diag()
        loss= torch.sum(loss)
        test_loss += loss
    
    test_loss /= len(valid_loader) # loss function already averages over batch size
    print('\nTest set: Average loss: ',test_loss)


In [103]:
for epoch in range(1, 200):
    train(epoch)
    test(epoch,valid_loader)

Train Epoch: 1 [0/3000 (0%)]	Loss: 205.767151
Train Epoch: 1 [640/3000 (21%)]	Loss: 177.903976
Train Epoch: 1 [1280/3000 (43%)]	Loss: 177.543259
Train Epoch: 1 [1920/3000 (64%)]	Loss: 169.389053
Train Epoch: 1 [2560/3000 (85%)]	Loss: 176.094849

Test set: Average loss:  Variable containing:
 10747.2432
[torch.FloatTensor of size 1]

Train Epoch: 2 [0/3000 (0%)]	Loss: 165.648041
Train Epoch: 2 [640/3000 (21%)]	Loss: 165.566315
Train Epoch: 2 [1280/3000 (43%)]	Loss: 153.664398
Train Epoch: 2 [1920/3000 (64%)]	Loss: 154.187103
Train Epoch: 2 [2560/3000 (85%)]	Loss: 138.759109

Test set: Average loss:  Variable containing:
 9155.5049
[torch.FloatTensor of size 1]

Train Epoch: 3 [0/3000 (0%)]	Loss: 139.212585
Train Epoch: 3 [640/3000 (21%)]	Loss: 127.501480
Train Epoch: 3 [1280/3000 (43%)]	Loss: 126.012360
Train Epoch: 3 [1920/3000 (64%)]	Loss: 134.932602
Train Epoch: 3 [2560/3000 (85%)]	Loss: 116.657761

Test set: Average loss:  Variable containing:
 8144.9194
[torch.FloatTensor of size 1

In [149]:
#Save model object
with open('model_200epochs.pkl', 'wb') as output:
    pickle.dump(model, output, pickle.HIGHEST_PROTOCOL)
#Code for reopen model later
# with open('model_200epochs.pkl', 'rb') as input:
#     model = pickle.load(input)

In [105]:
#Classification attempt
data,label = iter(valid_loader).next()

In [131]:
z_s = model(Variable(data,volatile=True)).data

In [116]:
train_loader_full = torch.utils.data.DataLoader(trainset_imoprt,batch_size=3000)

In [121]:
train_full_Tensor = iter(train_loader_full).next()[0]

In [125]:
train_full_Tensor = train_full_Tensor.view(3000,28*28)
train_full_labels = iter(train_loader_full).next()[1]

In [134]:
cosine_similarities_chart = torch.mm(z_s,train_full_Tensor.transpose(1,0)).numpy()

In [145]:
maximun_pos = np.apply_along_axis(lambda x: np.where(x==max(x)),1,cosine_similarities_chart).reshape(64)

In [147]:
train_full_labels.numpy()[maximun_pos]

array([5, 8, 8, 9, 0, 2, 6, 1, 8, 9, 8, 4, 5, 6, 5, 4, 4, 6, 1, 3, 3, 9, 0,
       8, 1, 0, 8, 8, 2, 9, 4, 5, 2, 1, 1, 9, 1, 5, 7, 0, 7, 2, 3, 0, 5, 4,
       5, 0, 2, 0, 2, 8, 8, 5, 7, 6, 2, 1, 8, 1, 0, 7, 7, 3])

In [148]:
label.numpy()

array([3, 8, 2, 9, 0, 2, 6, 1, 8, 4, 2, 4, 5, 6, 5, 4, 4, 6, 1, 3, 3, 9, 3,
       8, 1, 0, 8, 8, 2, 9, 4, 5, 2, 1, 1, 9, 1, 5, 7, 0, 7, 2, 3, 0, 5, 4,
       5, 0, 2, 0, 2, 5, 8, 5, 7, 6, 2, 1, 8, 1, 0, 7, 7, 3])